In [5]:
import pandas as pd
import pymysql
import re,json,os

In [6]:
base = os.environ['BUG_FREE_EUREKA_BASE']
pathTofile = os.path.join(base,'data','exercise06','E-TABM-713_A-MEXP-1551-analytics.tsv')

with open(os.path.join(base,'secrets.json')) as f:
    secrets = json.load(f)

conn = pymysql.connect(**secrets['test_db'])
cursor = conn.cursor(pymysql.cursors.DictCursor)

cursor.execute('create database if not exists biodb')


C:\Users\Sakthi\Anaconda2\envs\ipykernel_py3\lib\site-packages\pymysql\cursors.py:323: Warning: (1007, "Can't create database 'biodb'; database exists")
  self._do_get_result()


1

In [3]:
# load data to dataframe
df = pd.read_csv(pathTofile,sep='\t')
df.head()

,Gene ID,Gene Name,Design Element,g1_g2.p-value,g1_g2.t-statistic,g1_g2.log2foldchange
0,MIMAT0000062,MIMAT0000062,A_25_P00010086,0.702506,0.625362,0.078492
1,MIMAT0000062,MIMAT0000062,A_25_P00011584,0.975514,0.065110,0.007410
2,MIMAT0000063,MIMAT0000063,A_25_P00010070,0.139014,2.431636,0.556112
3,MIMAT0000063,MIMAT0000063,A_25_P00010071,0.049728,3.254492,0.811621
4,MIMAT0000064,MIMAT0000064,A_25_P00010072,0.098283,2.697813,0.757066


In [9]:
# create table from data frame

cursor.execute('use biodb')

#fix column names
df.columns = [re.sub('[ .-]','',x) for x in df.columns]

df.to_sql('dea',conn,flavor = 'mysql')

C:\Users\Sakthi\Anaconda2\envs\ipykernel_py3\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


ValueError: Table 'dea' already exists.

In [10]:
#check if table is created
cursor.execute("show tables")
cursor.fetchall()

[{'Tables_in_biodb': 'ache'},
 {'Tables_in_biodb': 'dea'},
 {'Tables_in_biodb': 'drugbank'},
 {'Tables_in_biodb': 'new_table_name2'},
 {'Tables_in_biodb': 'uniprot'}]

In [11]:
# to fetch first 10 lines from dea

cursor.execute("select * from dea limit 10")
cursor.fetchall()

[{'DesignElement': 'A_25_P00010086',
  'GeneID': 'MIMAT0000062',
  'GeneName': 'MIMAT0000062',
  'g1_g2log2foldchange': 0.0784915951074403,
  'g1_g2pvalue': 0.70250559386657,
  'g1_g2tstatistic': 0.625362183554288,
  'index': 0},
 {'DesignElement': 'A_25_P00011584',
  'GeneID': 'MIMAT0000062',
  'GeneName': 'MIMAT0000062',
  'g1_g2log2foldchange': 0.00741007045087905,
  'g1_g2pvalue': 0.975513746316429,
  'g1_g2tstatistic': 0.0651101242194695,
  'index': 1},
 {'DesignElement': 'A_25_P00010070',
  'GeneID': 'MIMAT0000063',
  'GeneName': 'MIMAT0000063',
  'g1_g2log2foldchange': 0.556111895299825,
  'g1_g2pvalue': 0.139013519150447,
  'g1_g2tstatistic': 2.43163612513429,
  'index': 2},
 {'DesignElement': 'A_25_P00010071',
  'GeneID': 'MIMAT0000063',
  'GeneName': 'MIMAT0000063',
  'g1_g2log2foldchange': 0.811621134771325,
  'g1_g2pvalue': 0.0497282355402996,
  'g1_g2tstatistic': 3.25449181729957,
  'index': 3},
 {'DesignElement': 'A_25_P00010072',
  'GeneID': 'MIMAT0000064',
  'GeneName':

In [13]:
# count the number of rows in the table

cursor.execute('select count(*) as no_of_rows from dea')
cursor.fetchall()

[{'no_of_rows': 1230}]

In [23]:
# average p_value for entire data

cursor.execute('select avg(g1_g2pvalue) as Average_p_value from dea')
cursor.fetchall()

[{'Average_p_value': 0.4529776486719517}]

In [26]:
# average p_value for each gene

cursor.execute('select GeneName ,avg(g1_g2pvalue) as Average_p_value from dea GROUP BY g1_g2pvalue limit 5')
cursor.fetchall()

[{'Average_p_value': None, 'GeneName': 'MIMAT0000079'},
 {'Average_p_value': 0.000321778898634328, 'GeneName': 'MIMAT0003227'},
 {'Average_p_value': 0.00145162156673662, 'GeneName': 'MIMAT0003304'},
 {'Average_p_value': 0.0022527138479248, 'GeneName': 'MIMAT0000082'},
 {'Average_p_value': 0.00253015446820666, 'GeneName': 'MIMAT0003227'}]

In [29]:
#gene with highest fold change lowest fold change

cursor.execute('select GeneName, g1_g2log2foldchange from dea where g1_g2log2foldchange = (select min(g1_g2log2foldchange) from dea)')
cursor.fetchall()

[{'GeneName': 'MIMAT0000451', 'g1_g2log2foldchange': -1.44215632547793}]

In [30]:
cursor.execute('select GeneName, g1_g2log2foldchange from dea where g1_g2log2foldchange = (select max(g1_g2log2foldchange) from dea)')
cursor.fetchall()

[{'GeneName': 'MIMAT0002177', 'g1_g2log2foldchange': 2.68544495738703}]